In [ ]:
overlay = True
max_labels_count = 500
max_lines_count = 500
max_boxes_count = 500
max_bars_back = 500
color_TRANSP_CSS = '#ffffff00'

#Settings

#General

mode = 'Historical' # 'Present'
style = 'Colored' # 'Monochrome'
show_trend = False

#Internal Structure
show_internals = True
show_ibull = 'All' # 'BOS' 'CHoCH'
swing_ibull_css = '#089981'

#Bear Structure
show_ibear = 'All' # 'BOS' 'CHoCH'
swing_ibear_css = '#f23645'
ifilter_confluence = False
internal_structure_size = 'Tiny' # 

#Swing Structure
show_Structure = True

#Bull Structure
show_bull = 'All' # 'BOS' 'CHoCH'
swing_bull_css = '#089981'

#Bear Structure
show_bear = 'All' # 'BOS' 'CHoCH'
swing_bear_css = '#f23645'
swing_structure_size = 'Small' # 'Tiny' 'Normal'

#Swings
show_swings = False

length = 50 #minval = 10
show_hl_swings = True

#Order Blocks
show_iob = True
iob_showlast = 5 #minval = 1
show_ob = False
ob_showlast = 5 #minval = 1
ob_filter = 'Atr' # 'Cumulative Mean Range'
ibull_ob_css = '#3179f5'
ibear_ob_css = '#f77c80'
bull_ob_css = '#1848cc'
bear_ob_css = '#b22833'

#EQH/EQL
show_eq = True
eq_len = 3 # minval = 1
eq_threshold = 0.1 # minval = 0 maxval = 0.5 step = 0.1
eq_size = 'Tiny'

#Fair Value Gaps
show_fvg = False
fvg_auto = True
fvg_tf = ''
bull_fvg_css = '#00ff68'
bear_fvg_css = '#ff0008'
fvg_extend = 1 # minval = 0

#Previous day/week high/low
#Daily
show_pdhl = False
pdhl_style = '⎯⎯⎯' # '----' '····'
pdhl_css = '#2157f3'

#Weekly
show_pwhl = False
pwhl_style = '⎯⎯⎯' # '----' '····'
pwhl_css = '#2157f3'

#Monthly
show_pmhl = False
pmhl_style = '⎯⎯⎯' # '----' '····'
pmhl_css = '#2157f3'

#Premium/Discount zones
show_sd = False
premium_css = '#f23645'
eq_css = '#b2b5be'
discount_css = '#089981'

#---------------------------------------------------------------------------
#Functions
n = bar_index #bar_index representa el numero de vela en la iteracion

atr = ta.atr(200)
cmean_range = ta.cum(high - low) / n

#HL Output function
def hl() => [high, low]

#Get ohlc values function
get_ohlc()=> [close[1], open[1], high, low, high[2], low[2]]

#Display Structure function
display_Structure(x, y, txt, css, dashed, down, lbl_size)=>
    structure_line = line.new(x, y, n, y
      , color = css
      , style = dashed ? line.style_dashed : line.style_solid)

    structure_lbl = label.new(int(math.avg(x, n)), y, txt
      , color = TRANSP_CSS
      , textcolor = css
      , style = down ? label.style_label_down : label.style_label_up
      , size = lbl_size)

    if mode == 'Present'
        line.delete(structure_line[1])
        label.delete(structure_lbl[1])

#Swings detection/measurements
swings(len)=>
    var os = 0
    
    upper = ta.highest(len)
    lower = ta.lowest(len)

    os := high[len] > upper ? 0 : low[len] < lower ? 1 : os[1]

    top = os == 0 and os[1] != 0 ? high[len] : 0
    btm = os == 1 and os[1] != 1 ? low[len] : 0

    [top, btm]

#Order block coordinates function
ob_coord(use_max, loc, target_top, target_btm, target_left, target_type)=>
    min = 99999999.
    max = 0.
    idx = 1

    ob_threshold = ob_filter == 'Atr' ? atr : cmean_range 

    #Search for highest/lowest high within the structure interval and get range
    if use_max
        for i = 1 to (n - loc)-1
            if (high[i] - low[i]) < ob_threshold[i] * 2
                max := math.max(high[i], max)
                min := max == high[i] ? low[i] : min
                idx := max == high[i] ? i : idx
    else
        for i = 1 to (n - loc)-1
            if (high[i] - low[i]) < ob_threshold[i] * 2
                min := math.min(low[i], min)
                max := min == low[i] ? high[i] : max
                idx := min == low[i] ? i : idx

    array.unshift(target_top, max)
    array.unshift(target_btm, min)
    array.unshift(target_left, time[idx])
    array.unshift(target_type, use_max ? -1 : 1)

#Set order blocks
display_ob(boxes, target_top, target_btm, target_left, target_type, show_last, swing, size)=>
    for i = 0 to math.min(show_last-1, size-1)
        get_box = array.get(boxes, i)

        box.set_lefttop(get_box, array.get(target_left, i), array.get(target_top, i))
        box.set_rightbottom(get_box, array.get(target_left, i), array.get(target_btm, i))
        box.set_extend(get_box, extend.right)

        color css = na
        
        if swing 
            if style == 'Monochrome'
                css := array.get(target_type, i) == 1 ? color.new(#b2b5be, 80) : color.new(#5d606b, 80)
                border_css = array.get(target_type, i) == 1 ? #b2b5be : #5d606b
                box.set_border_color(get_box, border_css)
            else
                css := array.get(target_type, i) == 1 ? bull_ob_css : bear_ob_css
                box.set_border_color(get_box, css)

            box.set_bgcolor(get_box, css)
        else
            if style == 'Monochrome'
                css := array.get(target_type, i) == 1 ? color.new(#b2b5be, 80) : color.new(#5d606b, 80)
            else
                css := array.get(target_type, i) == 1 ? ibull_ob_css : ibear_ob_css
            
            box.set_border_color(get_box, css)
            box.set_bgcolor(get_box, css)
        
#Line Style function
get_line_style(style) =>
    out = switch style
        '⎯⎯⎯'  => line.style_solid
        '----' => line.style_dashed
        '····' => line.style_dotted

#Set line/labels function for previous high/lows
phl(h, l, tf, css)=>
    var line high_line = line.new(na,na,na,na
      , xloc = xloc.bar_time
      , color = css
      , style = get_line_style(pdhl_style))

    var label high_lbl = label.new(na,na
      , xloc = xloc.bar_time
      , text = str.format('P{0}H', tf)
      , color = TRANSP_CSS
      , textcolor = css
      , size = size.small
      , style = label.style_label_left)

    var line low_line = line.new(na,na,na,na
      , xloc = xloc.bar_time
      , color = css
      , style = get_line_style(pdhl_style))

    var label low_lbl = label.new(na,na
      , xloc = xloc.bar_time
      , text = str.format('P{0}L', tf)
      , color = TRANSP_CSS
      , textcolor = css
      , size = size.small
      , style = label.style_label_left)

    hy = ta.valuewhen(h != h[1], h, 1)
    hx = ta.valuewhen(h == high, time, 1)

    ly = ta.valuewhen(l != l[1], l, 1)
    lx = ta.valuewhen(l == low, time, 1)

    if barstate.islast
        ext = time + (time - time[1])*20

        #High
        line.set_xy1(high_line, hx, hy)
        line.set_xy2(high_line, ext, hy)

        label.set_xy(high_lbl, ext, hy)

        #Low
        line.set_xy1(low_line, lx, ly)
        line.set_xy2(low_line, ext, ly)

        label.set_xy(low_lbl, ext, ly)